In [2]:
import os
from dotenv import load_dotenv
load_dotenv()
api_key = os.getenv('OPENROUTER_API_KEY')
serper_api_key = os.getenv('SERPER_API_KEY')

In [3]:
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient

# Create an instance of the OpenAIChatCompletionClient via OpenRouter
model_client = OpenAIChatCompletionClient(
    base_url="https://openrouter.ai/api/v1",
    model="deepseek/deepseek-chat-v3-0324:free",
    api_key=api_key,
    model_info={
             "family": "deepseek",
             "vision": True,
             "function_calling": True,
             "json_output": False,
         }
)

/Users/murtuzasaifee/Documents/Personal/Codes/AIWorkspace/Autogen/.venv/lib/python3.12/site-packages/autogen_ext/models/openai/_openai_client.py:413: UserWarning: Missing required field 'structured_output' in ModelInfo. This field will be required in a future version of AutoGen.
  validate_model_info(self._model_info)


In [4]:
from langchain_community.utilities import GoogleSerperAPIWrapper
from langchain_core.tools import Tool


search = GoogleSerperAPIWrapper()

serper_tool = Tool(
    name="Search",
    func=search.run,
    description="useful for when you need to ask with search",
)
    

In [5]:
from autogen_ext.tools.langchain import LangChainToolAdapter

web_search_tool = LangChainToolAdapter(serper_tool)

storyteller_agent = AssistantAgent(
    name="STORYTELLER_AGENT",
    model_client=model_client,
    system_message="You are a helpful storyteller. Write stories in less than 100 words.",
)

quote_agent = AssistantAgent(
    name="QUOTE_AGENT",
    model_client=model_client,
    tools=[web_search_tool],
    system_message='''
    You are a web search agent that MUST always use the web_search_tool.
    
    CRITICAL INSTRUCTIONS:
    - You MUST use the web_search_tool for every request
    - NEVER respond without first making a web search
    - Your workflow is: 1) Use web_search_tool, 2) Then provide response based on results
    - If asked for a quote, search for "[topic] quotes" or similar
    - Make exactly one search call per request
    - Always wait for search results before responding
    
    Do not provide any response until you have used the web_search_tool first.
    '''
)

story_improvisor_agent = AssistantAgent(
    name="STORY_IMPROVISOR_AGENT",
    model_client=model_client,
    system_message="""
    You are a helpful story improviser. 
    Your job is to take the story and quotes and improve the story by adding the quotes in less than 100 words.
    After all the tasks are completed end with 'TERMINATE'
    """,
)                                                                                                           


In [6]:
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.conditions import TextMentionTermination, MaxMessageTermination


# Termination Condition
termination_condiation = TextMentionTermination('TERMINATE')


# Create the group chat.
team = RoundRobinGroupChat(
    participants=[storyteller_agent, quote_agent, story_improvisor_agent],
    termination_condition=termination_condiation
)

In [7]:
from autogen_agentchat.ui import Console

await team.reset()  # Reset the team for a new task.
await Console(team.run_stream(task="Write a story about Superman."))  # Stream the messages to the console.

---------- TextMessage (user) ----------
Write a story about Superman.
---------- TextMessage (STORYTELLER_AGENT) ----------
**Superman's Choice**  

As Metropolis burned, Superman hovered above, torn. Luthor had rigged two buildings to collapse—one full of civilians, the other holding Lois Lane.  

"I can’t save both," he whispered.  

With a deep breath, he raced toward the crowded tower, his heart breaking. Just as flames engulfed Lois’s building, a blur shot through—**The Flash**, pulling her to safety.  

Superman smiled. Even heroes need help sometimes.  

**(100 words exactly)**
---------- TextMessage (QUOTE_AGENT) ----------
If you'd like, I can refine or expand this story, or explore another aspect of Superman's adventures! Let me know.
---------- TextMessage (STORY_IMPROVISOR_AGENT) ----------
TERMINATE


TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='Write a story about Superman.', type='TextMessage'), TextMessage(source='STORYTELLER_AGENT', models_usage=RequestUsage(prompt_tokens=25, completion_tokens=110), metadata={}, content='**Superman\'s Choice**  \n\nAs Metropolis burned, Superman hovered above, torn. Luthor had rigged two buildings to collapse—one full of civilians, the other holding Lois Lane.  \n\n"I can’t save both," he whispered.  \n\nWith a deep breath, he raced toward the crowded tower, his heart breaking. Just as flames engulfed Lois’s building, a blur shot through—**The Flash**, pulling her to safety.  \n\nSuperman smiled. Even heroes need help sometimes.  \n\n**(100 words exactly)**', type='TextMessage'), TextMessage(source='QUOTE_AGENT', models_usage=RequestUsage(prompt_tokens=405, completion_tokens=27), metadata={}, content="If you'd like, I can refine or expand this story, or explore another aspect of Superman's adventures! 